In [1]:
import cv2 as cv
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import torch
import argparse
from sklearn.metrics import classification_report, average_precision_score, f1_score, accuracy_score
from tqdm.notebook import tqdm_notebook
import bbox_visualizer as bbv
from yolov7_package import Yolov7Detector
import csv
import pandas as pd

In [2]:
labels = open('YOLOv7/classes.names').read().strip().split('\n')
model = Yolov7Detector(weights='yolov7-old/runs/train/yolov7-cvor9/weights/best.pt', img_size=[640, 480], classes='YOLOv7/classes.names')

Fusing layers... 
IDetect.fuse


/anaconda/envs/py38_default/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
def predict_image(image_filepath):
    try:
        classes, boxes, scores = model.detect(image_filepath)
        frame = image_filepath
    except:
        frame = cv.imread(image_filepath)
        classes, boxes, scores = model.detect(frame)
    img = model.draw_on_image(frame, boxes[0], scores[0], classes[0])
    cv.imwrite(f'models/results/{image_filepath}.jpg', frame)
    return classes[0]

In [4]:
y_pred, y_true = [], []
for file in tqdm_notebook(os.listdir('YOLOv7/test/images')):
    cur_pred = predict_image(f'YOLOv7/test/images/{file}')
    if (len(cur_pred)==2):
        y_pred.append(cur_pred)
    elif(len(cur_pred)>2):
        y_pred.append([cur_pred[0], cur_pred[1]])
    else:
        new_pred = y_pred.pop(len(y_pred)-1)
        y_pred.insert(len(y_pred)-1, new_pred)
        y_pred.insert(len(y_pred)-1, new_pred)
    labels_filename = file.split('.')[0]
    current_y_true = []
    with open (f'YOLOv7/test/labels/{labels_filename}.txt', 'r') as f:
        for line in f:
            current_y_true.append(int(line[0]))
    y_true.append(current_y_true)

  0%|          | 0/199 [00:00<?, ?it/s]

In [5]:
y_true = [sorted(element) for element in y_true]
y_pred = [sorted(element) for element in y_pred]

In [6]:
y_true = [item for sublist in y_true for item in sublist]
y_pred = [item for sublist in y_pred for item in sublist]

In [7]:
print(classification_report(y_true, y_pred, labels=[0,1,2,3,4,5,6,7]))
f1_macro = f1_score(y_true, y_pred, average='macro')
print(f'f1 macro: {f1_macro}')
print(f'Accuracy: {accuracy_score(y_true, y_pred)}')

              precision    recall  f1-score   support

           0       0.80      0.86      0.83        42
           1       0.00      0.00      0.00         1
           2       0.95      0.92      0.93       137
           3       0.60      0.30      0.40        10
           4       0.00      0.00      0.00         0
           5       0.75      0.97      0.85        62
           6       0.87      0.65      0.74        20
           7       0.93      0.89      0.91       126

   micro avg       0.88      0.88      0.88       398
   macro avg       0.61      0.57      0.58       398
weighted avg       0.88      0.88      0.88       398

f1 macro: 0.6656308873304708
Accuracy: 0.8793969849246231


/anaconda/envs/py38_default/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/py38_default/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/py38_default/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/py38_defa

# [mAP for Boxes](https://github.com/ZFTurbo/Mean-Average-Precision-for-Boxes)
#### y_true = ['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']
#### y_pred = ['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax']

In [8]:
"""
Author: Roman Solovyev, IPPM RAS
URL: https://github.com/ZFTurbo

Code based on: https://github.com/fizyr/keras-retinanet/blob/master/keras_retinanet/utils/eval.py
"""

def compute_overlap(boxes, query_boxes):
    """
    Args
        a: (N, 4) ndarray of float
        b: (K, 4) ndarray of float

    Returns
        overlaps: (N, K) ndarray of overlap between boxes and query_boxes
    """
    N = boxes.shape[0]
    K = query_boxes.shape[0]
    overlaps = np.zeros((N, K), dtype=np.float64)
    for k in range(K):
        box_area = (
            (query_boxes[k, 2] - query_boxes[k, 0]) *
            (query_boxes[k, 3] - query_boxes[k, 1])
        )
        for n in range(N):
            iw = (
                min(boxes[n, 2], query_boxes[k, 2]) -
                max(boxes[n, 0], query_boxes[k, 0])
            )
            if iw > 0:
                ih = (
                    min(boxes[n, 3], query_boxes[k, 3]) -
                    max(boxes[n, 1], query_boxes[k, 1])
                )
                if ih > 0:
                    ua = np.float64(
                        (boxes[n, 2] - boxes[n, 0]) *
                        (boxes[n, 3] - boxes[n, 1]) +
                        box_area - iw * ih
                    )
                    overlaps[n, k] = iw * ih / ua
    return overlaps


def get_real_annotations(table):
    res = dict()
    ids = table['ImageID'].values.astype('str')
    labels = table['LabelName'].values.astype('str')
    xmin = table['XMin'].values.astype(np.float32)
    xmax = table['XMax'].values.astype(np.float32)
    ymin = table['YMin'].values.astype(np.float32)
    ymax = table['YMax'].values.astype(np.float32)

    for i in range(len(ids)):
        id = ids[i]
        label = labels[i]
        if id not in res:
            res[id] = dict()
        if label not in res[id]:
            res[id][label] = []
        box = [xmin[i], ymin[i], xmax[i], ymax[i]]
        res[id][label].append(box)

    return res

def get_detections(table):
    res = dict()
    ids = table['ImageID'].values.astype('str')
    labels = table['LabelName'].values.astype('str')
    scores = table['Conf'].values.astype(np.float32)
    xmin = table['XMin'].values.astype(np.float32)
    xmax = table['XMax'].values.astype(np.float32)
    ymin = table['YMin'].values.astype(np.float32)
    ymax = table['YMax'].values.astype(np.float32)

    for i in range(len(ids)):
        id = ids[i]
        label = labels[i]
        if id not in res:
            res[id] = dict()
        if label not in res[id]:
            res[id][label] = []
        box = [xmin[i], ymin[i], xmax[i], ymax[i], scores[i]]
        res[id][label].append(box)

    return res

def _compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.

    Code originally from https://github.com/rbgirshick/py-faster-rcnn.

    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

def mean_average_precision_for_boxes(ann, pred, iou_threshold=0.5, exclude_not_in_annotations=False, verbose=True):
    """

    :param ann: path to CSV-file with annotations or numpy array of shape (N, 6)
    :param pred: path to CSV-file with predictions (detections) or numpy array of shape (N, 7)
    :param iou_threshold: IoU between boxes which count as 'match'. Default: 0.5
    :param exclude_not_in_annotations: exclude image IDs which are not exist in annotations. Default: False
    :param verbose: print detailed run info. Default: True
    :return: tuple, where first value is mAP and second values is dict with AP for each class.
    """

    if isinstance(ann, str):
        valid = pd.read_csv(ann)
    else:
        valid = pd.DataFrame(ann, columns=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax'])

    if isinstance(pred, str):
        preds = pd.read_csv(pred)
    else:
        preds = pd.DataFrame(pred, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

    ann_unique = valid['ImageID'].unique()
    preds_unique = preds['ImageID'].unique()

    if verbose:
        print('Number of files in annotations: {}'.format(len(ann_unique)))
        print('Number of files in predictions: {}'.format(len(preds_unique)))

    # Exclude files not in annotations!
    if exclude_not_in_annotations:
        preds = preds[preds['ImageID'].isin(ann_unique)]
        preds_unique = preds['ImageID'].unique()
        if verbose:
            print('Number of files in detection after reduction: {}'.format(len(preds_unique)))

    unique_classes = valid['LabelName'].unique().astype('str')# gabrielg changed astype(np.str)
    if verbose:
        print('Unique classes: {}'.format(len(unique_classes)))

    all_detections = get_detections(preds)
    all_annotations = get_real_annotations(valid)
    if verbose:
        print('Detections length: {}'.format(len(all_detections)))
        print('Annotations length: {}'.format(len(all_annotations)))
    print('-----------------')
    print(f'AP@{iou_threshold} per Class')
    print('-----------------')
    average_precisions = {}
    for zz, label in enumerate(sorted(unique_classes)):

        # Negative class
        if str(label) == 'nan':
            continue

        false_positives = []
        true_positives = []
        scores = []
        num_annotations = 0.0

        for i in range(len(ann_unique)):
            detections = []
            annotations = []
            id = ann_unique[i]
            if id in all_detections:
                if label in all_detections[id]:
                    detections = all_detections[id][label]
            if id in all_annotations:
                if label in all_annotations[id]:
                    annotations = all_annotations[id][label]

            if len(detections) == 0 and len(annotations) == 0:
                continue

            num_annotations += len(annotations)
            detected_annotations = []

            annotations = np.array(annotations, dtype=np.float64)
            for d in detections:
                scores.append(d[4])

                if len(annotations) == 0:
                    false_positives.append(1)
                    true_positives.append(0)
                    continue

                overlaps = compute_overlap(np.expand_dims(np.array(d, dtype=np.float64), axis=0), annotations)
                assigned_annotation = np.argmax(overlaps, axis=1)
                max_overlap = overlaps[0, assigned_annotation]

                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
                    false_positives.append(0)
                    true_positives.append(1)
                    detected_annotations.append(assigned_annotation)
                else:
                    false_positives.append(1)
                    true_positives.append(0)

        if num_annotations == 0:
            average_precisions[label] = 0, 0
            continue
        
        false_positives = np.array(false_positives)
        true_positives = np.array(true_positives)
        scores = np.array(scores)

        # sort by score
        indices = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives = true_positives[indices]

        # compute false positives and true positives
        false_positives = np.cumsum(false_positives)
        true_positives = np.cumsum(true_positives)

        # compute recall and precision
        recall = true_positives / num_annotations
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

        # compute average precision
        average_precision = _compute_ap(recall, precision)
        average_precisions[label] = average_precision, num_annotations
        if verbose:
            s1 = "{:30s} | {:.6f} | {:7d}".format(label, average_precision, int(num_annotations))
            print(s1)

    present_classes = 0
    precision = 0
    for label, (average_precision, num_annotations) in average_precisions.items():
        if num_annotations > 0:
            present_classes += 1
            precision += average_precision
    mean_ap = precision / present_classes
    if verbose:
        print('-----------------')
        print(f'mAP@{iou_threshold}: {mean_ap}')#gabrielg changed{:.6f}'.format(mean_ap))
        print('-----------------\n')
    return mean_ap, average_precisions
            

In [9]:
def predict_image_for_map_calculation(image_filepath):
    try:
        classes, boxes, scores = model.detect(image_filepath)
        frame = image_filepath
    except:
        frame = cv.imread(image_filepath)
        classes, boxes, scores = model.detect(frame)
    img = model.draw_on_image(frame, boxes[0], scores[0], classes[0])
    cv.imwrite(f'models/results/{image_filepath}.jpg', frame)
    return classes[0], boxes[0], scores[0]

In [12]:
def get_y_pred_and_y_true(iou_th):
    model = Yolov7Detector(weights='yolov7-old/runs/train/yolov7-cvor9/weights/best.pt', img_size=[416, 416], iou_thres=iou_th, classes='YOLOv7/classes.names')
    y_pred, y_true = [], []
    labels = open('YOLOv7/classes.names').read().strip().split('\n')
    print ('---------------------------------')
    print(f'Predicting for IoU Threshold={iou_th}')
    print ('---------------------------------')
    for file in tqdm_notebook(os.listdir('YOLOv7/test/images')):
        # normalize as in detect from yolov7_package
        frame = cv.imread(f'YOLOv7/test/images/{file}')
        
        img_shape = frame.shape
        w_img = img_shape[1]
        h_img = img_shape[0]
        
        classes, boxes, scores = predict_image_for_map_calculation(f'YOLOv7/test/images/{file}') 
        for i in range(len(classes)):
            xmin = boxes[i][0]#/dy if (boxes[i][0]<old_shape[1]) else 1.0
            ymin = boxes[i][1]#/dx if (boxes[i][1]<old_shape[0]) else 1.0
            xmax = boxes[i][2]#/dy if (boxes[i][2]<old_shape[1]) else 1.0
            ymax = boxes[i][3]#/dx if (boxes[i][3]<old_shape[0]) else 1.0
            class_id = classes[i]
            conf = scores[i]
            
            # ['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax']
            cur_pred = [file, labels[class_id], conf, xmin, xmax, ymin, ymax]
            y_pred.append(cur_pred)

        # y_true
        labels_filename = file.split('.')[0]
        with open (f'YOLOv7/test/labels/{labels_filename}.txt', 'r') as f:
            for line in f:
                line = line.split()
                
                class_id = int(line[0])
                
                x_cnt  = float(line[1])*w_img
                y_cnt  = float(line[2])*h_img
                w_bbox = float(line[3])*w_img
                h_bbox = float(line[4])*h_img                
                
                xmin = x_cnt - w_bbox/2
                ymin = y_cnt - h_bbox/2
                xmax = xmin  + w_bbox
                ymax = ymin  + h_bbox
                
                # ['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']
                cur_true = [file, labels[class_id], xmin, xmax, ymin, ymax]
                y_true.append(cur_true)

    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    iou_th_str = str(iou_th).split('.')[1]
    with open(f'mAP_for_boxes/results/y_pred_{iou_th_str}.csv', 'w', newline='') as f:
        f_write = csv.writer(f)
        f_write.writerow(["ImageID","LabelName","Conf","XMin","XMax","YMin","YMax"])
        for i in range(len(y_pred)):
            # ['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax']
            f_write.writerow([y_pred[i][0], y_pred[i][1], y_pred[i][2], y_pred[i][3], y_pred[i][4], y_pred[i][5], y_pred[i][6]])
        
    with open(f'mAP_for_boxes/results/y_true_{iou_th_str}.csv', 'w', newline='') as f:
        f_write = csv.writer(f)
        f_write.writerow(["ImageID","LabelName","XMin","XMax","YMin","YMax"])
        for i in range(len(y_true)):
            # ['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']
            f_write.writerow([y_true[i][0], y_true[i][1], y_true[i][2], y_true[i][3], y_true[i][4], y_true[i][5]])
        
    y_true = pd.read_csv(f'mAP_for_boxes/results/y_true_{iou_th_str}.csv')
    y_pred = pd.read_csv(f'mAP_for_boxes/results/y_pred_{iou_th_str}.csv')
    y_true = y_true[['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']].values
    y_pred = y_pred[['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax']].values
    
    return y_true, y_pred

In [13]:
for iou_th in [0.25, 0.5, 0.75]:
    y_true, y_pred = get_y_pred_and_y_true(iou_th)
    mean_ap, average_precisions = mean_average_precision_for_boxes(y_true, y_pred, iou_threshold=iou_th)

Fusing layers... 
IDetect.fuse
---------------------------------
Predicting for IoU Threshold=0.25
---------------------------------


  0%|          | 0/199 [00:00<?, ?it/s]

Number of files in annotations: 199
Number of files in predictions: 199
Unique classes: 7
Detections length: 199
Annotations length: 199
-----------------
AP@0.25 per Class
-----------------
Left_Empty                     | 0.000000 |     126
Left_Forceps                   | 0.000000 |      62
Left_Needle_driver             | 0.000000 |      10
Left_Scissors                  | 0.000000 |       1
Right_Empty                    | 0.005556 |      20
Right_Needle_driver            | 0.001335 |     137
Right_Scissors                 | 0.000000 |      42
-----------------
mAP@0.25: 0.000984325603720807
-----------------

Fusing layers... 
IDetect.fuse
---------------------------------
Predicting for IoU Threshold=0.5
---------------------------------


  0%|          | 0/199 [00:00<?, ?it/s]

Number of files in annotations: 199
Number of files in predictions: 199
Unique classes: 7
Detections length: 199
Annotations length: 199
-----------------
AP@0.5 per Class
-----------------
Left_Empty                     | 0.000000 |     126
Left_Forceps                   | 0.000000 |      62
Left_Needle_driver             | 0.000000 |      10
Left_Scissors                  | 0.000000 |       1
Right_Empty                    | 0.000000 |      20
Right_Needle_driver            | 0.000000 |     137
Right_Scissors                 | 0.000000 |      42
-----------------
mAP@0.5: 0.0
-----------------

Fusing layers... 
IDetect.fuse
---------------------------------
Predicting for IoU Threshold=0.75
---------------------------------


  0%|          | 0/199 [00:00<?, ?it/s]

Number of files in annotations: 199
Number of files in predictions: 199
Unique classes: 7
Detections length: 199
Annotations length: 199
-----------------
AP@0.75 per Class
-----------------
Left_Empty                     | 0.000000 |     126
Left_Forceps                   | 0.000000 |      62
Left_Needle_driver             | 0.000000 |      10
Left_Scissors                  | 0.000000 |       1
Right_Empty                    | 0.000000 |      20
Right_Needle_driver            | 0.000000 |     137
Right_Scissors                 | 0.000000 |      42
-----------------
mAP@0.75: 0.0
-----------------

